In [1]:
import pandas as pd
from pathlib import Path

In [40]:
index_no = 2

In [41]:
df_old = pd.read_csv(f"labels_{index_no}_old.csv", dtype=str)
print(df_old.shape)
df_old.head()

(3104, 5)


,id,pattern,update_date,label,para
0,2110.11575,"github, data repos",04/21/22,0,Make sure the target repo (the repo to be anal...
1,2103.15335,dataset,04/21/22,0,3.1 Datasets
2,2109.12907,dataset,04/21/22,0,Table 3: Usage of relations in the dataset.
3,2012.04966,data,NaN,NaN,"In one thesis [27], we aimed for generalizatio..."
4,2010.0562,data,NaN,NaN,"[16] Elena Parkhomenko, David Tritchler, and J..."


In [42]:
df_new = pd.read_csv(f"labels_{index_no}_new.csv", dtype=str).drop(columns=["update_date", "label"])
print(df_new.shape)
df_new.head()

(2912, 4)


,id,pattern,token_count,para
0,1702.03196,data,397,"eaveragedstructuredperceptron(Collins,2002)fro..."
1,1702.03196,github,15,"SivaReddy,OscarT¨ackstr¨om,MichaelCollins,TomK..."
2,1702.03196,github,265,6.QuestionAnsweringonFreebaseviaRelationExtrac...
3,1702.03196,github,421,UniversalSemanticParsingSivaReddy†∗OscarT¨acks...
4,1702.03196,github,555,1.1WithOriginalTypeSystemWewillﬁrstattempttode...


In [43]:
df_new = df_new.merge(df_old, on=["id", "pattern", "para"], how="left")
df_new = df_new[["id", "pattern", "token_count", "update_date", "label", "para"]].sort_values(by=["id", "pattern", "token_count"])
print(df_new.shape)
df_new.head()


(2912, 6)


,id,pattern,token_count,update_date,label,para
0,1702.03196,data,397,NaN,NaN,"eaveragedstructuredperceptron(Collins,2002)fro..."
1,1702.03196,github,15,NaN,NaN,"SivaReddy,OscarT¨ackstr¨om,MichaelCollins,TomK..."
2,1702.03196,github,265,NaN,NaN,6.QuestionAnsweringonFreebaseviaRelationExtrac...
3,1702.03196,github,421,NaN,NaN,UniversalSemanticParsingSivaReddy†∗OscarT¨acks...
4,1702.03196,github,555,NaN,NaN,1.1WithOriginalTypeSystemWewillﬁrstattempttode...


In [44]:
df_new.to_csv(f"labels_{index_no}_new_merged.csv", index=False)

In [4]:
def merge_existing_new_labels(df_existing, df_new):
    df_existing = df_existing.merge(
        df_new[["para", "id", "pattern", "token_count"]], on=["para", "id", "token_count"], how="outer"
    )

    # if pattern_y is NaN, then copy pattern_x to pattern_y
    df_existing["pattern"] = df_existing["pattern_y"].fillna(df_existing["pattern_x"])

    # drop columns that are not needed, pattern_x and pattern_y
    df_existing = df_existing.drop(["pattern_x", "pattern_y"], axis=1)
    return df_existing[["id", "pattern", "token_count", "update_date", "label", "para"]]
